# **SUPERSTORE SALES PREDICTION**

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('train.csv')

In [3]:
data

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,9796,CA-2017-125920,21/05/2017,28/05/2017,Standard Class,SH-19975,Sally Hughsby,Corporate,United States,Chicago,Illinois,60610.0,Central,OFF-BI-10003429,Office Supplies,Binders,"Cardinal HOLDit! Binder Insert Strips,Extra St...",3.7980
9796,9797,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,OFF-AR-10001374,Office Supplies,Art,"BIC Brite Liner Highlighters, Chisel Tip",10.3680
9797,9798,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,TEC-PH-10004977,Technology,Phones,GE 30524EE4,235.1880
9798,9799,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,TEC-PH-10000912,Technology,Phones,Anker 24W Portable Micro USB Car Charger,26.3760


In [4]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

## **Preprocessing**

In [5]:
def encode_dates(df, column):
    df = df.copy()
    df[column] = pd.to_datetime(df[column])
    df[column + '_year'] = df[column].apply(lambda x: x.year)
    df[column + '_month'] = df[column].apply(lambda x: x.month)
    df[column + '_day'] = df[column].apply(lambda x: x.day)
    df = df.drop(column, axis=1)
    return df

def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [6]:
def preprocess_inputs(df):
    df = df.copy()

    # Drop unnecessary columns
    df = df.drop(['Row ID', 'Customer Name', 'Country', 'Product Name'], axis=1)

    # Drop customer-specific feature columns
    df = df.drop(['Order ID', 'Customer ID'], axis=1)

    # Extract date features
    df = encode_dates(df, column='Order Date')
    df = encode_dates(df, column='Ship Date')

    # One-hot encode categorical features
    for column in ['Ship Mode', 'Segment', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category']:
        df = onehot_encode(df, column=column)

    # Split df into X and y
    y = df['Sales']
    X = df.drop('Sales', axis=1)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)

    # Scale X
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)

    return X_train, X_test, y_train, y_test

In [8]:
def encode_dates(df, column):
    df = df.copy()
    df[column] = pd.to_datetime(df[column], format='%d/%m/%Y') # Specify the correct date format
    df[column + '_year'] = df[column].apply(lambda x: x.year)
    df[column + '_month'] = df[column].apply(lambda x: x.month)
    df[column + '_day'] = df[column].apply(lambda x: x.day)
    df = df.drop(column, axis=1)
    return df

In [10]:
X_train, X_test, y_train, y_test = preprocess_inputs(data)

In [11]:
X_train

,Order Date_year,Order Date_month,Order Date_day,Ship Date_year,Ship Date_month,Ship Date_day,Ship Mode_First Class,Ship Mode_Same Day,Ship Mode_Second Class,Ship Mode_Standard Class,...,Sub-Category_Envelopes,Sub-Category_Fasteners,Sub-Category_Furnishings,Sub-Category_Labels,Sub-Category_Machines,Sub-Category_Paper,Sub-Category_Phones,Sub-Category_Storage,Sub-Category_Supplies,Sub-Category_Tables
0,1.153020,-0.551120,-0.061954,1.138089,-0.522895,0.348732,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,-0.400194,-0.314653,-0.30333,-0.13789,-0.179443
1,-0.626244,0.665707,-0.290575,-0.638920,0.674688,0.121500,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,-0.400194,-0.314653,-0.30333,-0.13789,-0.179443
2,1.153020,-0.246913,-1.662298,1.138089,-0.223499,-0.901046,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,2.498785,-0.314653,-0.30333,-0.13789,-0.179443
3,-0.626244,0.969914,0.509597,-0.638920,0.974084,1.030429,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,-0.400194,-0.314653,-0.30333,-0.13789,-0.179443
4,1.153020,0.361501,-1.205057,1.138089,0.375292,-0.787430,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,2.498785,-0.314653,-0.30333,-0.13789,-0.179443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6855,1.153020,0.665707,0.852528,1.138089,0.674688,1.484894,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,-0.400194,3.178101,-0.30333,-0.13789,-0.179443
6856,0.263388,-1.767947,-0.176264,0.249584,-1.720479,-0.105733,2.326202,-0.249166,-0.484918,-1.212556,...,-0.162742,-0.145387,3.064953,-0.187924,-0.108625,-0.400194,-0.314653,-0.30333,-0.13789,-0.179443
6857,0.263388,-1.159534,-0.862126,0.249584,-1.121687,-0.446581,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,2.498785,-0.314653,-0.30333,-0.13789,-0.179443
6858,-1.515875,0.969914,1.424080,-1.527425,1.273480,-1.696359,-0.429885,-0.249166,2.062203,-1.212556,...,-0.162742,-0.145387,3.064953,-0.187924,-0.108625,-0.400194,-0.314653,-0.30333,-0.13789,-0.179443


In [12]:
y_train

8788    698.352
847     287.940
3397     25.920
6244    119.040
8198     15.552
         ...   
2895     35.880
7813     10.560
905      12.960
5192    397.600
235     617.976
Name: Sales, Length: 6860, dtype: float64

## **Training**

In [13]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))
x = tf.keras.layers.Dense(256, activation='relu')(inputs)
x = tf.keras.layers.Dense(256, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3102)]            0         
                                                                 
 dense (Dense)               (None, 256)               794368    
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dense_2 (Dense)             (None, 1)                 257       
                                                                 
Total params: 860417 (3.28 MB)
Trainable params: 860417 (3.28 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [14]:
model.compile(
    optimizer='adam',
    loss='mse'
)

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau()
    ]
)

Epoch 1/100
172/172 [==============================] - 4s 19ms/step - loss: 486767.7188 - val_loss: 185773.9219 - lr: 0.0010
Epoch 2/100
172/172 [==============================] - 3s 20ms/step - loss: 391083.0000 - val_loss: 161167.0625 - lr: 0.0010
Epoch 3/100
172/172 [==============================] - 3s 18ms/step - loss: 351409.4375 - val_loss: 300489.1875 - lr: 0.0010
Epoch 4/100
172/172 [==============================] - 3s 15ms/step - loss: 264937.4688 - val_loss: 1308503.8750 - lr: 0.0010
Epoch 5/100
172/172 [==============================] - 3s 15ms/step - loss: 220808.0938 - val_loss: 4801602.5000 - lr: 0.0010
Epoch 6/100
172/172 [==============================] - 3s 15ms/step - loss: 161491.4219 - val_loss: 18868416.0000 - lr: 0.0010
Epoch 7/100
172/172 [==============================] - 4s 22ms/step - loss: 126263.3906 - val_loss: 34179472.0000 - lr: 0.0010


## **Results**

In [15]:
test_loss = model.evaluate(X_test, y_test, verbose=0)

print("Test Loss: {:.5f}".format(test_loss))

Test Loss: 214659.21875


In [16]:
y_pred = np.squeeze(model.predict(X_test))
test_r2 = r2_score(y_test, y_pred)

print("Test R^2 Score: {:.5f}".format(test_r2))

92/92 [==============================] - 1s 4ms/step
Test R^2 Score: 0.25871
